In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

2024-12-30 22:32:29.962062: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735565549.988125 1704750 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735565549.996198 1704750 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-30 22:32:30.022379: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
n_class=10
img_siz=(32, 32, 3)

patch_size=4
p2=(img_siz[0]//patch_size)**2
d_model=64
h=8
N=6

In [3]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.p_siz=patch_size
        
    def call(self, img):
        batch_size=tf.shape(img)[0]
        patches=tf.image.extract_patches(images=img,sizes=[1,self.p_siz,self.p_siz,1],strides=[1,self.p_siz,self.p_siz,1],rates=[1,1,1,1],padding='VALID')
        patch_dims=patches.shape[-1]
        patches=tf.reshape(patches, [batch_size,-1,patch_dims])
        return patches
    
class PatchEncoder(layers.Layer):
    def __init__(self, p2, d_model):
        super(PatchEncoder, self).__init__()
        self.p2=p2
        self.projection=layers.Dense(units=d_model)
        self.position_embedding=layers.Embedding(input_dim=p2, output_dim=d_model)
        
    def call(self, patch):
        positions=tf.range(start=0, limit=self.p2, delta=1)
        encoded=self.projection(patch)+self.position_embedding(positions)
        return encoded

In [4]:
def create_vit_classifier():
    input=layers.Input(shape=(img_siz))
    nor=layers.Normalization()(input)
    
    patches=Patches(patch_size)(nor)
    x=PatchEncoder(p2, d_model)(patches)
    
    for _ in range(N):
        x1=layers.LayerNormalization(epsilon=1e-6)(x)
        x2=layers.MultiHeadAttention(num_heads=h, key_dim=d_model//h, dropout=0.1)(x1, x1)
        x3=layers.Add()([x2, x])
        x4=layers.LayerNormalization(epsilon=1e-6)(x3)
        x5=layers.Dense(d_model*2, activation=tf.nn.gelu)(x4)
        x6=layers.Dropout(0.1)(x5)
        x7=layers.Dense(d_model, activation=tf.nn.gelu)(x6)
        x8=layers.Dropout(0.1)(x7)
        x=layers.Add()([x8, x3])
        
        x=layers.LayerNormalization(epsilon=1e-6)(x)
        x=layers.Flatten()(x)
        x=layers.Dropout(0.5)(x)
        x=layers.Dense(2048, activation=tf.nn.gelu)(x)
        x=layers.Dropout(0.5)(x)
        x=layers.Dense(1024, activation=tf.nn.gelu)(x)
        x=layers.Dropout(0.5)(x)
        output=layers.Dense(n_class, activation='softmax')(x)
        
        model=keras.Model(inputs=input, outputs=output)
        return model

In [6]:
model=create_vit_classifier()
model.layers[1].adapt(x_train)

model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(from_logits=False), metrics=['accuracy'])
hist=model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test), verbose=1)

res=model.evaluate(x_test, y_test, verbose=0)
print('accuracy=', res[1]*100)

Epoch 1/100


E0000 00:00:1735565607.925071 1704947 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
E0000 00:00:1735565608.440516 1704947 cuda_dnn.cc:522] Loaded runtime CuDNN library: 9.1.0 but source was compiled with: 9.3.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-12-30 22:33:28.461082: W tensorflow/core/framework/op_kernel.cc:1841] OP_REQUIRES failed at xla_ops.cc:577 : FAILED_PRECONDITION: DNN library initialization failed. Look at the er

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/bj/transformer/.wv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/bj/transformer/.wv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/bj/transformer/.wv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/bj/transformer/.wv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 604, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/home/bj/transformer/.wv/lib/python3.11/site-packages/ipykernel/kernelbase.py", li